# Fairness and Ethics Basics - Responsible Predictive Analytics

<hr>

<center>
<div>
<img src="https://raw.githubusercontent.com/davi-moreira/2026Summer_predictive_analytics_purdue_MGMT474/main/notebooks/figures/mgmt_474_ai_logo_02-modified.png" width="200"/>
</div>
</center>

# <center><a class="tocSkip"></center>
# <center>MGMT47400 Predictive Analytics</center>
# <center>Professor: Davi Moreira </center>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/davi-moreira/2026Summer_predictive_analytics_purdue_MGMT474/blob/main/notebooks/17_fairness_slicing_model_cards.ipynb)

---

## Learning Objectives

By the end of this notebook, you will be able to:

1. Identify fairness risks and ethical failure modes in predictive systems
2. Compute basic group fairness diagnostics (when sensitive attributes exist)
3. Use slicing to detect performance disparities across segments
4. Write a model card-style limitations and responsible-use section
5. Apply responsible AI framing to the course project deliverable

---

## 1. Setup

This cell loads the standard data-wrangling and plotting libraries plus
scikit-learn metrics we will need for sliced evaluation. We lock
`RANDOM_SEED = 474` so that the synthetic data generation and group
assignments are identical every time the notebook is run, making all
fairness comparisons fully reproducible.


In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import warnings

warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

RANDOM_SEED = 474
np.random.seed(RANDOM_SEED)

print("✓ Setup complete!")

**Reading the output:**

The `Setup complete!` line confirms that all imports -- including
`accuracy_score`, `precision_score`, `recall_score`, `f1_score`, and
`confusion_matrix` -- loaded without error. The random seed is fixed at
**474**, which means the synthetic group assignments and train/test splits
will be identical on every run.

**Why this matters:** Reproducibility is especially important in fairness
analysis because small changes in the random split can flip the direction
of a disparity, making conclusions unreliable.

---


## 2. Fairness Vocabulary

### Key Concepts:

**Disparity**: Systematic difference in outcomes across groups

**Harm**: Negative impact on individuals or groups
- **Allocation harm**: Withholding opportunities or resources
- **Quality-of-service harm**: Worse performance for some groups
- **Representational harm**: Stereotyping or denigration

**Proxy variables**: Features correlated with sensitive attributes

**Feedback loops**: Model decisions influence future data, potentially amplifying bias

### Common Fairness Metrics:

1. **Demographic Parity**: Equal selection rate across groups  
   P(ŷ=1 | A=a) = P(ŷ=1 | A=b)

2. **Equal Opportunity**: Equal true positive rate across groups  
   P(ŷ=1 | y=1, A=a) = P(ŷ=1 | y=1, A=b)

3. **Equalized Odds**: Equal TPR and FPR across groups

⚠️ **Important**: These metrics often conflict with each other!

---

## 3. Generate Synthetic Data with Group Attribute

We use `make_classification` to create 2,000 samples with 15 features
(10 informative, 5 redundant) and a 60/40 positive-negative class split.
A synthetic `group` column is added **after** feature generation -- it is
randomly assigned and deliberately **not** used as a training feature. This
setup mirrors real-world scenarios where a sensitive attribute exists in the
data but is excluded from the model, yet performance may still differ across
groups.


In [ ]:
# Generate classification data
X, y = make_classification(
    n_samples=2000, n_features=15, n_informative=10,
    n_redundant=5, weights=[0.6, 0.4],
    random_state=RANDOM_SEED
)

# Add synthetic "group" attribute (not used in training)
# Group A: 60%, Group B: 40%
group = np.random.choice(['Group_A', 'Group_B'], size=len(X), p=[0.6, 0.4])

# Create DataFrame
feature_names = [f'feature_{i}' for i in range(X.shape[1])]
df = pd.DataFrame(X, columns=feature_names)
df['target'] = y
df['group'] = group

print(f"Dataset shape: {df.shape}")
print(f"\nGroup distribution:")
print(df['group'].value_counts())
print(f"\nTarget distribution by group:")
print(pd.crosstab(df['group'], df['target'], normalize='index'))

**Reading the output:**

The dataset has **2,000 rows** and **17 columns** (15 features + target +
group). The group distribution should be approximately **60 % Group_A** and
**40 % Group_B**, matching the probabilities we set. The cross-tabulation of
target by group shows whether the positive-class base rate differs between
groups -- if it does, even a perfect model will produce different selection
rates, which is the root of many fairness dilemmas.

**Key takeaway:** Understanding the base rate by group *before* modelling is
essential. A difference in base rates does not automatically mean the model
is unfair, but it does mean that equal selection rates and equal accuracy
cannot both be achieved simultaneously.

---


## 4. Train Model (Without Using Group Attribute)

We train a Random Forest on the 15 numeric features only -- the `group`
column is deliberately excluded. The train/test split is stratified by group
so both sets contain proportional representation. After training, we compute
overall accuracy, precision, recall, and F1 on the test set as a baseline
before drilling into group-level performance.


In [ ]:
# Split data (stratified by group to ensure representation)
train_df, test_df = train_test_split(df, test_size=0.3, stratify=df['group'], random_state=RANDOM_SEED)

# Separate features, target, and group
X_train = train_df[feature_names]
y_train = train_df['target']
group_train = train_df['group']

X_test = test_df[feature_names]
y_test = test_df['target']
group_test = test_df['group']

# Train model WITHOUT group attribute
model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=RANDOM_SEED, n_jobs=-1)
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:, 1]

print("\n=== OVERALL MODEL PERFORMANCE ===")
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(f"Precision: {precision_score(y_test, y_pred):.4f}")
print(f"Recall: {recall_score(y_test, y_pred):.4f}")
print(f"F1: {f1_score(y_test, y_pred):.4f}")

**Reading the output:**

The overall **accuracy**, **precision**, **recall**, and **F1** are computed
on the full test set without any group breakdown. These numbers set the
baseline: they tell us how the model performs *on average*. A strong overall
F1 does not guarantee that both groups are served equally well -- that is
exactly what the sliced evaluation in the next section will reveal.

**Why this matters:** Reporting only aggregate metrics is a common pitfall
in responsible AI. Stakeholders deserve to know whether performance is
uniform across the populations the model will affect.

---


## 5. Slice-Based Performance Evaluation

### 5.1 Performance by Group

Aggregate metrics can mask important disparities. Slicing the test set by
`group` and computing accuracy, precision, recall, and F1 for each segment
separately reveals whether the model serves all groups equally. Even when
overall accuracy looks strong, one group may suffer noticeably lower recall
or precision.


In [ ]:
# Compute metrics by group
slice_results = []

for grp in ['Group_A', 'Group_B']:
    mask = group_test == grp
    y_true_grp = y_test[mask]
    y_pred_grp = y_pred[mask]
    
    slice_results.append({
        'Group': grp,
        'Sample_Size': len(y_true_grp),
        'Accuracy': accuracy_score(y_true_grp, y_pred_grp),
        'Precision': precision_score(y_true_grp, y_pred_grp, zero_division=0),
        'Recall': recall_score(y_true_grp, y_pred_grp, zero_division=0),
        'F1': f1_score(y_true_grp, y_pred_grp, zero_division=0)
    })

slice_df = pd.DataFrame(slice_results)

print("\n=== PERFORMANCE BY GROUP ===")
print(slice_df.to_string(index=False))

# Compute disparities
print("\n=== PERFORMANCE DISPARITIES ===")
for metric in ['Accuracy', 'Precision', 'Recall', 'F1']:
    gap = slice_df.loc[0, metric] - slice_df.loc[1, metric]
    print(f"{metric} Gap (A - B): {gap:+.4f}")

**Reading the output:**

The **Performance by Group** table shows accuracy, precision, recall, and F1
for Group_A and Group_B side by side. The **Performance Disparities** section
computes the signed gap (A minus B) for each metric. A positive gap means
Group_A scores higher; a negative gap means Group_B does. Pay special
attention to the **Recall gap**: it tells you whether qualified individuals
in one group are more likely to be missed.

**Key takeaway:** Even small metric gaps (e.g., 0.03 in recall) can compound
into large cumulative effects when the model runs on thousands of cases.
Always compute both the gap magnitude and the sample sizes to assess
practical significance.

---


In [ ]:
# Visualize performance gaps
metrics = ['Accuracy', 'Precision', 'Recall', 'F1']

fig, ax = plt.subplots(figsize=(10, 6))
x = np.arange(len(metrics))
width = 0.35

ax.bar(x - width/2, slice_df.loc[0, metrics], width, label='Group A', alpha=0.8)
ax.bar(x + width/2, slice_df.loc[1, metrics], width, label='Group B', alpha=0.8)

ax.set_xlabel('Metric')
ax.set_ylabel('Score')
ax.set_title('Model Performance by Group')
ax.set_xticks(x)
ax.set_xticklabels(metrics)
ax.legend()
ax.set_ylim([0, 1])
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

**Reading the output:**

The grouped bar chart plots accuracy, precision, recall, and F1 side by
side for Group_A and Group_B. Bars of similar height indicate comparable
performance; a visible height difference on any metric flags a potential
disparity worth investigating. Because the y-axis runs from 0 to 1, even a
bar-height difference of a few percentage points is easy to spot.

**Why this matters:** Visualisations make disparities tangible for
non-technical stakeholders. A chart that shows one group's recall bar
noticeably shorter than the other communicates the issue far more
effectively than a table of numbers.

---


## 📝 PAUSE-AND-DO Exercise 1 (5 minutes)

**Task:** Create slice performance table and highlight one disparity (if any).

**Instructions:**
1. Review the slice performance results above
2. Identify the largest performance gap between groups
3. Assess whether this gap is meaningful/concerning
4. Propose one hypothesis for why the gap exists

---

### YOUR DISPARITY ANALYSIS HERE:

**Largest Disparity:**  
[Which metric and how large?]

**Is it Concerning?:**  
[Your assessment with reasoning]

**Hypothesis:**  
[Why might this disparity exist?]

---

## 6. Fairness Metrics

### 6.1 Selection Rate (Demographic Parity)

In [ ]:
# Compute selection rates
fairness_metrics = []

for grp in ['Group_A', 'Group_B']:
    mask = group_test == grp
    y_pred_grp = y_pred[mask]
    y_true_grp = y_test[mask]
    
    # Selection rate
    selection_rate = y_pred_grp.mean()
    
    # Confusion matrix
    tn, fp, fn, tp = confusion_matrix(y_true_grp, y_pred_grp).ravel()
    
    # Rates
    tpr = tp / (tp + fn) if (tp + fn) > 0 else 0  # True Positive Rate
    fpr = fp / (fp + tn) if (fp + tn) > 0 else 0  # False Positive Rate
    
    fairness_metrics.append({
        'Group': grp,
        'Selection_Rate': selection_rate,
        'TPR': tpr,
        'FPR': fpr
    })

fairness_df = pd.DataFrame(fairness_metrics)

print("\n=== FAIRNESS METRICS ===")
print(fairness_df.to_string(index=False))

# Compute fairness gaps
print("\n=== FAIRNESS GAPS ===")
sr_gap = fairness_df.loc[0, 'Selection_Rate'] - fairness_df.loc[1, 'Selection_Rate']
tpr_gap = fairness_df.loc[0, 'TPR'] - fairness_df.loc[1, 'TPR']
fpr_gap = fairness_df.loc[0, 'FPR'] - fairness_df.loc[1, 'FPR']

print(f"Selection Rate Gap: {sr_gap:+.4f}")
print(f"TPR Gap (Equal Opportunity): {tpr_gap:+.4f}")
print(f"FPR Gap: {fpr_gap:+.4f}")

print("\n⚠️ Interpretation:")
print("  - Selection Rate: Are both groups selected at similar rates?")
print("  - TPR Gap: Do qualified individuals have equal chance across groups?")
print("  - FPR Gap: Are false positives distributed equally?")

**Reading the output:**

Three fairness diagnostics are printed. The **Selection Rate** per group
measures demographic parity: if both groups are selected (predicted positive)
at the same rate, the Selection Rate Gap is near zero. The **TPR gap**
captures equal opportunity: whether truly positive individuals are caught
at equal rates regardless of group. The **FPR gap** rounds out equalized
odds: whether truly negative individuals are falsely flagged at equal rates.

**Key takeaway:** These three metrics often conflict -- reducing the
selection-rate gap may widen the TPR gap, and vice versa. Document which
metric your organisation prioritises and why.

---


## 7. Model Card Template

### 7.1 Model Card Structure

A Model Card is a short, structured document that accompanies a trained model
and discloses its intended use, training data, performance metrics,
limitations, and ethical considerations. The template below follows the
format proposed by Mitchell et al. (2019). Filling it out forces you to
think beyond accuracy and confront potential harms before deployment.


# Model Card: [Your Model Name]

## Model Details
- **Model Type**: Random Forest Classifier
- **Version**: 1.0
- **Date**: June 9, 2027
- **Developed By**: [Your name/team]

## Intended Use
**Primary Use Cases:**
- [Describe primary intended application]
- [Context where model should be used]

**Out-of-Scope Uses:**
- [Contexts where model should NOT be used]
- [Scenarios where model may fail or cause harm]

## Training Data
- **Source**: [Dataset name and source]
- **Size**: [Number of samples]
- **Time Period**: [When data was collected]
- **Geography**: [Where data was collected]
- **Preprocessing**: [Key preprocessing steps]

## Evaluation Data
- **Source**: [Test set details]
- **Size**: [Number of samples]
- **Split Method**: [How test set was created]

## Metrics
**Overall Performance:**
- Accuracy: X.XX
- Precision: X.XX
- Recall: X.XX
- F1: X.XX

**Performance by Group:**
- Group A: [Metrics]
- Group B: [Metrics]
- Performance Gap: [Description]

## Limitations
1. **Data Limitations**:
   - [Describe data quality issues, biases, or gaps]
   - [Temporal or geographic limitations]

2. **Model Limitations**:
   - [Known failure modes or weak segments]
   - [Assumptions that may not hold]
   - [Performance disparities across groups]

3. **Deployment Limitations**:
   - [Contexts where model should not be used]
   - [Required human oversight]
   - [Monitoring requirements]

## Ethical Considerations
**Potential Harms:**
- [Allocation harms - who might be denied opportunities?]
- [Quality-of-service harms - who might receive worse predictions?]
- [Feedback loop risks - how might the model affect future data?]

**Mitigation Strategies:**
- [What steps are taken to reduce harm?]
- [How are performance disparities addressed?]
- [What oversight mechanisms exist?]

## Recommendations
1. **Usage Recommendations**:
   - [How should model outputs be used?]
   - [What human review is required?]

2. **Monitoring**:
   - [What metrics should be tracked over time?]
   - [How often should model be re-evaluated?]

3. **Update Schedule**:
   - [When should model be retrained?]
   - [What triggers retraining?]

---

## 📝 PAUSE-AND-DO Exercise 2 (5 minutes)

**Task:** Draft a model card limitations section (6-8 lines).

**Instructions:**
1. Review the model card template above
2. Write a limitations section for your project model
3. Include data limitations, model limitations, and ethical considerations
4. Be specific and evidence-based

---

### YOUR MODEL CARD LIMITATIONS SECTION:

## Limitations

1. **Data Limitations**:  
[Your text here - be specific]

2. **Model Limitations**:  
[Your text here - reference specific performance issues]

3. **Ethical Considerations**:  
[Your text here - discuss potential harms]

---

## 8. Wrap-Up: Key Takeaways

### What We Learned Today:

1. **Fairness Vocabulary**: Disparity, harm, proxies, feedback loops
2. **Slice Evaluation**: Performance analysis by demographic groups
3. **Fairness Metrics**: Selection rate, TPR/FPR gaps, equalized odds
4. **Model Cards**: Structured documentation of model details and limitations
5. **Responsible Communication**: Honest disclosure of limitations and risks

### Responsible AI Best Practices:

- ✓ Always evaluate performance across relevant demographic groups
- ✓ Document known limitations and failure modes
- ✓ Consider both allocation and quality-of-service harms
- ✓ Be cautious about fairness claims - trade-offs are inevitable
- ✓ Plan for monitoring and regular re-evaluation
- ✓ Involve diverse stakeholders in evaluation and deployment decisions

### Remember:

> **"Perfect fairness is impossible - but honest documentation of limitations is essential."**  
> Focus on transparency, not perfection.

---

## Bibliography

- Barocas, S., Hardt, M., & Narayanan, A. (2019). *Fairness and Machine Learning*. [fairmlbook.org](http://fairmlbook.org/)
- Hardt, M., Price, E., & Srebro, N. (2016). "Equality of Opportunity in Supervised Learning." *NeurIPS*.
- Mitchell, M., et al. (2019). "Model Cards for Model Reporting." *FAT**.
- Chouldechova, A. (2017). "Fair prediction with disparate impact." *Big Data*.
- Selbst, A.D., et al. (2019). "Fairness and Abstraction in Sociotechnical Systems." *FAT**.

---




<center>

Thank you!

</center>